In [6]:
from games_webscrape import scrape_game_data
from team_data_gather_good import get_team_data
import pandas as pd
from multiprocessing import Process
import time
import numpy as np

In [7]:
endpoints = ['points-per-game', 'opponent-points-per-game']
SCRAPE_TEAM_DATA = True

In [8]:
if SCRAPE_TEAM_DATA:
    t0 = time.time()
    data_df = get_team_data(2010, 2020, endpoints)
    t1 = time.time()
    print(f"this took {round(t1-t0,2)} seconds.")
    data_df.to_pickle('data_df_2010-2019.pickle')

[DatetimeIndex(['2009-11-01', '2009-11-02', '2009-11-03', '2009-11-04',
               '2009-11-05', '2009-11-06', '2009-11-07', '2009-11-08',
               '2009-11-09', '2009-11-10',
               ...
               '2010-03-22', '2010-03-23', '2010-03-24', '2010-03-25',
               '2010-03-26', '2010-03-27', '2010-03-28', '2010-03-29',
               '2010-03-30', '2010-03-31'],
              dtype='datetime64[ns]', length=151, freq='D'), DatetimeIndex(['2010-11-01', '2010-11-02', '2010-11-03', '2010-11-04',
               '2010-11-05', '2010-11-06', '2010-11-07', '2010-11-08',
               '2010-11-09', '2010-11-10',
               ...
               '2011-03-22', '2011-03-23', '2011-03-24', '2011-03-25',
               '2011-03-26', '2011-03-27', '2011-03-28', '2011-03-29',
               '2011-03-30', '2011-03-31'],
              dtype='datetime64[ns]', length=151, freq='D'), DatetimeIndex(['2012-01-01', '2012-01-02', '2012-01-03', '2012-01-04',
               '2012-01-05

In [37]:
data_df

points-per-game opponent-points-per-game
2009-11-01 Atlanta               112.7                    118.6
           Boston                112.4                    108.1
           Brooklyn              112.0                    112.3
           Charlotte             110.9                    111.9
           Chicago               105.1                    113.4
...                                ...                      ...
2019-03-31 Sacramento            114.1                    114.8
           San Antonio           111.9                    110.3
           Toronto               114.3                    108.4
           Utah                  111.2                    106.2
           Washington            114.4                    117.2

[43500 rows x 2 columns]

In [41]:
games_df

,Date,Home,Away,DidHomeWin
0,2015-11-01,Charlotte,Atlanta,False
1,2015-11-01,Boston,San Antonio,False
2,2015-11-01,Miami,Houston,True
3,2015-11-01,Toronto,Milwaukee,True
4,2015-11-01,Chicago,Orlando,True
...,...,...,...,...
11479,2013-03-31,New Orleans,Cleveland,True
11480,2013-03-31,Washington,Toronto,True
11481,2013-03-31,Chicago,Detroit,True
11482,2013-03-31,San Antonio,Miami,False


In [9]:
data_df = pd.read_pickle('data_df_2010-2019.pickle')

In [10]:
t0 = time.time()
games_df = scrape_game_data(2010, 2020)
t1 = time.time()
print(f"this took {round(t1-t0,2)} seconds.")


[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2016, 2017, 2018, 2019]
this took 16.99 seconds.


In [11]:
games_df.query('Home == "Okla City"').Home.count()


390

In [57]:
data_df.loc[('2015-11-01', 'Charlotte')]

points-per-game             110.9
opponent-points-per-game    111.9
Name: (2015-11-01 00:00:00, Charlotte), dtype: object

In [45]:
training_data = []
games_df.loc[0].Date.date()

for i in range(len(games_df)):
    try:
        game = games_df.loc[i]

        homeTeam, awayTeam = str(game.Home), str(game.Away)
        date_str = str(game.Date.date())   
        home_index = (date_str, homeTeam) # Index to get the stats for a team on a certain date from data_df
        away_index = (date_str, awayTeam)
        home_stats = data_df.loc[home_index]
        away_stats = data_df.loc[away_index]
        training_data.append([ [list(home_stats), list(away_stats)] , game.DidHomeWin]) # [1, 0 ] if AWAY WON, and [0,1] if HOME WON$
    except KeyError as e:
        print(date_str) 
        print(home_index)
    
training_data = np.array(training_data, dtype=object)
training_data

2013-12-31
('2013-12-31', 'Boston')
2013-12-31
('2013-12-31', 'Indiana')
2013-12-31
('2013-12-31', 'Orlando')
2013-12-31
('2013-12-31', 'Houston')
2013-12-31
('2013-12-31', 'San Antonio')
2013-12-31
('2013-12-31', 'Chicago')
2013-12-31
('2013-12-31', 'Okla City')
2013-12-31
('2013-12-31', 'LA Lakers')


array([[list([['110.9', '111.9'], ['112.7', '118.6']]), False],
       [list([['112.4', '108.1'], ['111.9', '110.3']]), False],
       [list([['105.6', '105.6'], ['112.9', '109.4']]), True],
       ...,
       [list([['105.1', '113.4'], ['107.5', '107.5']]), True],
       [list([['111.9', '110.3'], ['105.6', '105.6']]), False],
       [list([['104.8', '114.2'], ['112.4', '108.1']]), True]],
      dtype=object)

In [76]:
training_data[0][1]

False

In [14]:
np.random.shuffle(training_data)
len(training_data)

11476

In [17]:
# np.save("training_data_2010-2019.npy", training_data)

In [36]:
# training_data = np.load('training_data_2010-2019.npy', allow_pickle=True)
# training_data

array([[[list(['107.5', '107.5']), list(['107.9', '104.3'])],
        [1.0, 0.0]],

       [[list(['117.5', '108.1']), list(['117.5', '111.8'])],
        [0.0, 1.0]],

       [[list(['105.1', '113.4']), list(['107.5', '107.5'])],
        [1.0, 0.0]],

       ...,

       [[list(['107.9', '104.3']), list(['117.5', '108.1'])],
        [1.0, 0.0]],

       [[list(['114.3', '110.4']), list(['104.8', '114.2'])],
        [0.0, 1.0]],

       [[list(['107.0', '116.3']), list(['104.5', '113.8'])],
        [0.0, 1.0]]], dtype=object)

In [70]:
def determine_pythag_expectation(points_for, points_against):
    win = (float(points_for)**(13.91))/(float(points_for)**13.91 + float(points_against)**13.91)
    return win


In [81]:
correct = 0
extremly_wrong = 0
for data in training_data:
    home_expectation = determine_pythag_expectation(data[0][0][0][0], data[0][0][0][1])
    away_expectation = determine_pythag_expectation(data[0][0][1][0], data[0][0][1][1])
    if (home_expectation >= away_expectation) and data[0][1]:
        correct += 1
    else:
        extremly_wrong += 1

print(correct)
print(extremly_wrong)
print(correct/(correct+extremly_wrong))

9177
2299
0.7996688741721855


In [71]:
home_expectation = determine_pythag_expectation(training_data[0][0][0][0], training_data[0][0][0][1])

In [72]:
home_expectation

0.46882394928586535

In [ ]:
# building the conv net


In [25]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [27]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.linear(4, 64)
        self.fc2 = nn.linear(64, 64)
        self.fc3 = nn.linear(64, 64)
        self.fc4 = nn.linear(64, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        return x

In [30]:
net = Net()

In [32]:
X = torch.randn((2,2))
X.view(-1, 4)

tensor([[-0.5302, -0.8876,  0.1972,  0.2840]])

In [33]:
tensor_array = torch.from_numpy(training_data)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, int64, int32, int16, int8, uint8, and bool.